In [5]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python import PythonOperator
import pandas as pd
import mysql.connector

In [48]:
stocks1 = pd.read_csv('./ETFs_stocks_dataset/stocks/A.csv')
stocks2 = pd.read_csv('./ETFs_stocks_dataset/etfs/AAAU.csv')
symbol = pd.read_csv('./ETFs_stocks_dataset/symbols_valid_meta.csv')

,Date,Open,High,Low,Close,Adj Close,Volume


In [50]:
symbol = symbol.loc[:,['Symbol', 'Security Name']]

In [63]:
symbol['Symbol'].values([1])

TypeError: 'numpy.ndarray' object is not callable

In [69]:
stocks1['Date'] = stocks1['Date'].astype('string')

Date          string
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [70]:
stocks2

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-08-15,11.840000,11.840000,11.74,11.740000,11.740000,27300
1,2018-08-16,11.780000,11.800000,11.74,11.740000,11.740000,428400
2,2018-08-17,11.800000,11.820000,11.77,11.820000,11.820000,52400
3,2018-08-20,11.880000,11.910000,11.85,11.900000,11.900000,28700
4,2018-08-21,11.920000,11.950000,11.89,11.930000,11.930000,30600
...,...,...,...,...,...,...,...
405,2020-03-26,16.450001,16.530001,16.23,16.400000,16.400000,212400
406,2020-03-27,16.350000,16.379999,16.25,16.290001,16.290001,172500
407,2020-03-30,16.290001,16.330000,16.09,16.240000,16.240000,536900
408,2020-03-31,16.000000,16.090000,15.71,15.720000,15.720000,193700


In [21]:
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 4, 29),
    'retries': 1
}

dag = DAG(
    'etf_stock_data',
    default_args=default_args,
    description='Setup a data structure to retain all data from ETFs and stocks',
    schedule=None
)

def read_data():
    # Read data from CSV file
    data = stocks1

    # Connect to MySQL database
    cnx = mysql.connector.connect(user='root', host='localhost')
    
    # Create a new database
    cursor = cnx.cursor()
    cursor.execute('CREATE DATABASE IF NOT EXISTS mydatabase')
    cnx.database = 'mydatabase'

    # Insert data into database table
    for index, row in data.iterrows():
        cursor.execute('INSERT INTO etf_stock_data (Symbol, SecurityName, Date, Open, High, Low, Close, AdjClose, Volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)', (row['Symbol'], row['Security Name'], row['Date'], row['Open'], row['High'], row['Low'], row['Close'], row['Adj Close'], row['Volume']))
        cnx.commit()

    # Close database connection
    cursor.close()
    cnx.close()

read_data_task = PythonOperator(
    task_id='read_data',
    python_callable=read_data,
    dag=dag
)

In [32]:
for index, row in stocks1.iterrows():
    print(row)

Date         1999-11-18
Open          32.546494
High          35.765381
Low           28.612303
Close         31.473534
Adj Close     27.068665
Volume         62546300
Name: 0, dtype: object
Date         1999-11-19
Open           30.71352
High          30.758226
Low           28.478184
Close         28.880543
Adj Close     24.838577
Volume         15234100
Name: 1, dtype: object
Date         1999-11-22
Open          29.551144
High          31.473534
Low           28.657009
Close         31.473534
Adj Close     27.068665
Volume          6577800
Name: 2, dtype: object
Date         1999-11-23
Open          30.400572
High          31.205294
Low           28.612303
Close         28.612303
Adj Close      24.60788
Volume          5975600
Name: 3, dtype: object
Date         1999-11-24
Open          28.701717
High          29.998211
Low           28.612303
Close         29.372318
Adj Close     25.261524
Volume          4843200
Name: 4, dtype: object
Date         1999-11-26
Open          29.2381

Date         2012-02-13
Open          31.967096
High          31.974249
Low           31.130186
Close         31.359085
Adj Close     28.624168
Volume          4791600
Name: 3077, dtype: object
Date         2012-02-14
Open          31.287554
High          31.652361
Low           31.001431
Close         31.316166
Adj Close     28.584991
Volume          3659400
Name: 3078, dtype: object
Date         2012-02-15
Open          31.831188
High          31.831188
Low           31.065809
Close         31.216022
Adj Close     28.493586
Volume          3490500
Name: 3079, dtype: object
Date         2012-02-16
Open          30.758226
High          31.437769
Low           29.871244
Close         31.351931
Adj Close     28.617638
Volume          9901700
Name: 3080, dtype: object
Date         2012-02-17
Open          31.502146
High          31.573677
Low           30.743919
Close          30.85837
Adj Close     28.167118
Volume          5687300
Name: 3081, dtype: object
Date         2012-02-21
Open  

In [22]:
sql_f = read_data()

KeyError: 'Symbol'